In [1]:
import numpy as np 
import os
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
path1 = []
path2 = []
path3 = []
path0 = []
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Non Demented'):
    for filename in filenames:
        path0.append(os.path.join(dirname, filename))
        
print('First done')
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Mild Dementia'):
    for filename in filenames:
        path1.append(os.path.join(dirname, filename))
        
print('Second done')
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Moderate Dementia'):
    for filename in filenames:
        path3.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input/imagesoasis/Data/Very mild Dementia'):
    for filename in filenames:
        path2.append(os.path.join(dirname, filename))

First done
Second done


In [3]:
4+6

10

In [4]:
def create_dict(file_names):
    img_dict = {}
    for name in file_names:
        ind = name[:-8]
        if ind in img_dict:
            img_dict[ind].append(name)
        else:
            dem = name[31:34]
            if dem=='Non':
                img_dict[ind] = [0]
            elif dem == 'Mil':
                img_dict[ind] = [1]
            elif dem == 'Ver':
                img_dict[ind] = [2]
            else:
                img_dict[ind] = [3]
            img_dict[ind].append(name)
    return img_dict
img_dict = create_dict(path0 + path1 + path2 + path3)

In [5]:
def create_df_from_dict(img_dict):
    df = pd.DataFrame({k: v[0] for k,v in img_dict.items()}, index = ['Class']).T
    return df
df_all = create_df_from_dict(img_dict)
df_all.head()

,Class
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0302_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0114_MR1_mpr-1,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0150_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0253_MR1_mpr-3,0
/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0349_MR1_mpr-4,0


In [6]:
# Assuming df is your DataFrame and 'class' is your target feature
X = df_all.drop('Class', axis=1)
y = df_all['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
def create_test_train(img_dict, y_train, y_test):
    X_train_n = []
    y_train_n = []
    X_test_n = []
    y_test_n = []
    for ind in y_test.index:
        paths = img_dict[ind]
        imgs = []
        for j in range(1,len(paths)):
            img = Image.open(paths[j]).resize((128,128))
            img = np.array(img)
            imgs.append(img)
            #y_test_n.append(y_test[ind])
        X_test_n.append(imgs)
        
    print('first part done')
    for ind in y_train.index:
        paths = img_dict[ind]
        imgs = []
        for j in range(1,len(paths)):
            img = Image.open(paths[j]).resize((128,128))
            img = np.array(img)
            imgs.append(img)
            #y_train_n.append(y_train[ind])
        X_train_n.append(imgs)
    
    
    return X_train_n, X_test_n, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = create_test_train(img_dict, y_train, y_test)

first part done


In [9]:
# import torch
# from torch import nn
# import torch.nn.functional as F

# # Assuming each image is 128x128x3
# input_shape = (128, 128, 3)

# # Define the CNN model
# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.flatten = nn.Flatten()

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.flatten(x)
#         return x

# # Create 61 CNN models for the 61 input images
# cnn_models = [CNNModel() for _ in range(61)]

# # Create 61 inputs for the 61 input images
# inputs = [torch.randn(input_shape).permute(2,0,1) for _ in range(61)]

# # Extract features from each input using the corresponding CNN model
# features = [model(inp) for model, inp in zip(cnn_models, inputs)]

# # Concatenate all features together
# concatenated = torch.cat(features, dim=1)

# # Add a dense layer for final prediction
# output_layer = nn.Linear(concatenated.size(1), 4)
# output = output_layer(concatenated)

# # Create the model
# model = nn.ModuleList([*cnn_models, output_layer])

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())

# print(model)

In [10]:
# for id_images, label in zip(X_train, y_train):
#     outputs = model(*id_images)
#     loss = nn.CrossEntropyLoss(outputs.unsqueeze(0), label)

# # Switch to evaluation mode
# model.eval()

# # No need to track gradients for testing
# with torch.no_grad():
#     correct_predictions = 0
#     total_predictions = 0

#     for id_images, label in zip(X_test, y_test):
#         outputs = model(*id_images)
#         _, predicted = torch.max(outputs.data, 1)
#         total_predictions += label.size(0)
#         correct_predictions += (predicted == label).sum().item()

#     print('Test Accuracy: %d %%' % (100 * correct_predictions / total_predictions))


In [11]:
import torch
x_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
x_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))

In [12]:
x_train.shape

torch.Size([1133, 61, 128, 128, 3])

In [13]:
print('Time to run the network')

Time to run the network


In [14]:
del path0
del path1
del path2
del path3
del X_train, X_test

In [15]:
del df_all, img_dict

In [16]:
del X, y

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


In [22]:

from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Assuming x_train, y_train, x_test, y_test are your data
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Change 1 to 3
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.rnn = nn.GRU(64*32*32, 256, num_layers=2, batch_first=True)
        self.fc = nn.Linear(256, 4)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        x = x.view(batch_size * timesteps, C, H, W)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(batch_size, timesteps, -1)  # Use reshape instead of view
        x, _ = self.rnn(x)
        x = x[:, -1, :]
        x = torch.sigmoid(self.fc(x))
        return x



# Instantiate the network
model = Net().to(device)

# Define the loss function and the optimizer
criterion = nn.BCELoss()
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr = lr)


# Initialize empty lists to store losses
train_losses = []
test_losses = []

# Training loop
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # Set the model to training mode
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        inputs = inputs.permute(0,1,4,2,3)
        # Convert labels to one-hot encoding
        labels_onehot = torch.zeros(labels.size(0), 4, device = device)
        labels_onehot.scatter_(1, labels.view(-1,1), 1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels_onehot)
        loss.backward()
        optimizer.step()
        # Compute training loss
        train_losses.append(loss.item())
        # print statistics
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader)}')

    # Compute test loss
    #test_loss = 0.0
    #model.eval()  # Set the model to evaluation mode
    #with torch.no_grad():  # Do not compute gradients for these operations
    #    for i, data in enumerate(test_dataloader, 0):
    #        inputs, labels = data
    #        inputs = inputs.float()
    #        inputs = inputs.permute(0,1,4,2,3)
    #        labels_onehot = torch.zeros(labels.size(0), 4,device = device)
    #        labels_onehot.scatter_(1, labels.view(-1,1), 1)
    #        outputs = model(inputs)
    #        loss = criterion(outputs, labels_onehot)
    #        test_loss += loss.item()
    #        test_losses.append(test_loss / len(test_dataloader))  # Store the test loss
    #print(f'Epoch {epoch+1}, test loss: {test_loss/len(test_dataloader)}')

print('Finished Training')



Epoch 1, training loss: 0.3337417563630475
Epoch 1, test loss: 0.3103690461979972
Epoch 2, training loss: 0.30825647670361733
Epoch 2, test loss: 0.31183818313810563
Epoch 3, training loss: 0.3101722101370494
Epoch 3, test loss: 0.3062293595737881
Epoch 4, training loss: 0.3063674428396755
Epoch 4, test loss: 0.3121926734844844
Epoch 5, training loss: 0.3100707820720143
Epoch 5, test loss: 0.3129917085170746
Epoch 6, training loss: 0.307182014402416
Epoch 6, test loss: 0.3063543852832582
Epoch 7, training loss: 0.30885093038280803
Epoch 7, test loss: 0.3064671771393882
Epoch 8, training loss: 0.3078494978447755
Epoch 8, test loss: 0.30964656008614433
Epoch 9, training loss: 0.3129097914530171
Epoch 9, test loss: 0.30779213706652325
Epoch 10, training loss: 0.31033505251010257
Epoch 10, test loss: 0.3100193540255229
Finished Training


In [23]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms

# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image, label = self.images[idx], self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Assuming x_train, y_train, x_test, y_test are your data
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

train_dataset = MyDataset(x_train, y_train, transform=transform)
test_dataset = MyDataset(x_test, y_test, transform=transforms.ToTensor())


# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the neural network with dropout
class Net(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(p=dropout_prob)  # Dropout layer
        self.rnn = nn.GRU(64*32*32, 256, num_layers=2, batch_first=True)
        self.fc = nn.Linear(256, 4)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        x = x.view(batch_size * timesteps, C, H, W)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(batch_size, timesteps, -1)
        x, _ = self.rnn(x)
        x = x[:, -1, :]
        x = torch.sigmoid(self.fc(x))
        return x

# Instantiate the network and move it to the device
model = Net().to(device)

# Define the loss function and the optimizer
criterion = nn.BCELoss()
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# Initialize empty lists to store losses
train_losses = []
test_losses = []

# Training loop
for epoch in range(10):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        inputs = inputs.permute(0, 1, 4, 2, 3)
        labels_onehot = torch.zeros(labels.size(0), 4, device=device)
        labels_onehot.scatter_(1, labels.view(-1, 1), 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels_onehot)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader)}')

    
print('Finished Training')


10

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    raw_outputs = model(x_test.float().permute(0,1,4,2,3))  # Raw output (logits) from the model
    predicted_probs = torch.sigmoid(raw_outputs)  # Apply sigmoid to get probabilities


# Assuming you have predictions (predicted_probs) and ground truth labels (y_test)
threshold = 0.5  # Set your desired threshold for converting probabilities to binary labels
predicted_labels = (predicted_probs > threshold).astype(int)

# Compute multilabel confusion matrix
mcm = multilabel_confusion_matrix(y_test, predicted_labels)

# Print confusion matrix for each class
for i, label in enumerate(["Class 0", "Class 1", "Class 2", "Class 3"]):
    print(f"Confusion matrix for {label}:\n{mcm[i]}")

# Compute classification report
print("\nClassification Report:")
print(classification_report(y_test, predicted_labels))
